Instructions for running the below code: Go to the following website: http://web.mta.info/developers/developer-data-terms.html#data <br>
Accept the terms and conditions, and you will get to a new page with data feeds. Scroll to the Static Data Feeds, and under GTFS, click on and download the link titled 'New York City Transit Subway'. This zip file will have three useful files: <br>
1) stops.txt will list the stop_id for every station on the line. Use this to come up with the appropriate stop_id. <br>
2) stop_times.txt lists the time each scheduled trip is set to stop at each stop in the system. Use the path to this file as stops_path in the below code. <br>
3) trips.txt lists the route_id that corresponds to each trip. Use the path to this file as trips_path in the below code.

In [181]:
import numpy as np
import pandas as pd

def get_station_schedule(stops_path,trips_path,stop_id): #get all scheduled stops at a station with given stop_id, split by direction
                #can also input a specific direction with stop id, and will just get the schedule for that direction
    master_schedule=pd.read_csv(stops_path,delimiter=',')
    trips_df=pd.read_csv(trips_path,delimiter=',')
    master_schedule=master_schedule.join(trips_df.set_index('trip_id'),on='trip_id')
    if (('N' in stop_id) or ('S' in stop_id)):        
        single_platform_sched=master_schedule[master_schedule['stop_id']==stop_id]
        return [single_platform_sched]         #gets schedule for a single specified direction in a specified station
    else:
        single_station_sched_N=master_schedule[master_schedule['stop_id']==stop_id+'N']
        single_station_sched_S=master_schedule[master_schedule['stop_id']==stop_id+'S']
        return [single_station_sched_N, single_station_sched_S]

def sort_by_day(df):    
    sunday_trips=[]
    weekday_trips=[]
    saturday_trips=[]
    for entry in df['trip_id']:
        if ('Sunday' in entry):
            sunday_trips.append(entry)
        elif ('Weekday' in entry):
            weekday_trips.append(entry)
        elif ('Saturday' in entry):
            saturday_trips.append(entry)
    sunday_schedule=df[df['trip_id'].isin(sunday_trips)] #only the entries that have a trip id in sunday_trips
    weekday_schedule=df[df['trip_id'].isin(weekday_trips)]
    saturday_schedule=df[df['trip_id'].isin(saturday_trips)]
    trips={}
    trips['Sunday']=sunday_schedule
    trips['Weekday']=weekday_schedule
    trips['Saturday']=saturday_schedule
    return trips

def get_all_schedules_by_day(stops_path,trips_path,stop_id):
    schedules=[sort_by_day(item) for item in get_station_schedule(stops_path,trips_path,stop_id)]
    return schedules

In [184]:
#Example for running code
#stops_path=r"C:\Users\Raayan\Documents\Columbia\Summer2020\DesignChallenge\schedule_info\stop_times.txt"
#trips_path=r"C:\Users\Raayan\Documents\Columbia\Summer2020\DesignChallenge\schedule_info\trips.txt"
#stop_id='120'
#things=get_all_schedules_by_day(stops_path,trips_path,stop_id)
#things

[{'Sunday':                                       trip_id arrival_time departure_time  \
  58     AFA19GEN-1037-Sunday-00_007200_1..N03R     01:40:30       01:40:30   
  704    AFA19GEN-1037-Sunday-00_009200_1..N03R     02:00:30       02:00:30   
  742    AFA19GEN-1037-Sunday-00_011200_1..N03R     02:20:30       02:20:30   
  780    AFA19GEN-1037-Sunday-00_013200_1..N03R     02:40:30       02:40:30   
  818    AFA19GEN-1037-Sunday-00_015200_1..N03R     03:00:30       03:00:30   
  ...                                       ...          ...            ...   
  94324  AFA19GEN-3041-Sunday-00_137600_3..N01R     23:44:30       23:44:30   
  94358  AFA19GEN-3041-Sunday-00_139000_3..N01R     23:58:30       23:58:30   
  94392  AFA19GEN-3041-Sunday-00_140400_3..N01R     24:12:30       24:12:30   
  94401  AFA19GEN-3041-Sunday-00_143800_3..N42R     24:04:00       24:04:00   
  94410  AFA19GEN-3041-Sunday-00_146400_3..N42R     24:30:00       24:30:00   
  
        stop_id  stop_sequence  stop_he